# Path explorer

This notebook contains code related to the definition and exploration of the different states that the `Solar field (SF) with Thermal storage (TS)` can take at different samples in time.

- Directed graph build by defining nodes / vertices that are the states that the FSM of the system can take, and repeated at each time step. Also, the edges (representing transitions between states) connecting these nodes are obtained from the FSM itself, so only valid transitions from the states at each step are considered.
- Path exploration. In order to obtain every possible path for any given initial state and a prediction horizon (max number of steps to evaluate). A Deep First Search (DFS) is performed in a recursive function to find every possibility. The result is a list with every possible path or trajectory for the given initial states. Note that increasing the number of steps can lead to the "tree explosion" due to the exponential growth that takes place in these kind of systems.
- The visualization of these two aspects, is then visualized.

Expected result:

![expected result](docs/attachments/sf_ts_binary_tree.png)

In [1]:
import pandas as pd
import numpy as np
from loguru import logger
import json

%load_ext autoreload

%autoreload 2

from solarMED_modeling.fsms import SolarFieldWithThermalStorage_FSM
from solarMED_modeling import SF_TS_State
from solarMED_optimization.path_exploration import (
    Node, 
    generate_edges,
    generate_edges_dataframe
)

Np = 5 # Prediction horizon


In [2]:
# Build states/nodes/vertices dataframe

base_list = [str(state.value) for state in SF_TS_State]
N_nodes = Np*len(base_list)

# Just the SF-TS to test
# Prepend 'step{idx}_' to each string in the base list, repeat N times
result = [
    Node(
        step_idx=step_idx,
        state=state,
    ).model_dump()
    for step_idx in range(Np) for state in [state for state in SF_TS_State]
]

nodes_df = pd.DataFrame(result)

display(nodes_df.head())


,step_idx,state,node_id,state_value,state_name,x_pos,y_pos
0,0,SF_TS_State.IDLE,step000_00,00,IDLE,0,0
1,0,SF_TS_State.RECIRCULATING_TS,step000_01,01,RECIRCULATING_TS,0,1
2,0,SF_TS_State.HEATING_UP_SF,step000_10,10,HEATING_UP_SF,0,2
3,0,SF_TS_State.SF_HEATING_TS,step000_11,11,SF_HEATING_TS,0,3
4,1,SF_TS_State.IDLE,step001_00,00,IDLE,1,0


In [3]:
# Build transition/connection/edges dataframe

step_idx = 0

edges_list = []
for step_idx in range(Np):
    edges_list = generate_edges(edges_list, step_idx, system='SFTS', Np=Np)
    
# Convert to dataframe
edges_df = generate_edges_dataframe(edges_list)

display(edges_df.head())

,src_node_id,dst_node_id,step_idx,src_name,dst_name,transition_id,line_type,x_pos_src,x_pos_dst,y_pos_src,y_pos_dst
0,step000_00,step001_01,0,IDLE,RECIRCULATING_TS,step000_start_recirculating_ts,solid,None,None,None,None
1,step000_00,step001_10,0,IDLE,HEATING_UP_SF,step000_start_recirculating_sf,solid,None,None,None,None
2,step000_00,step001_00,0,IDLE,IDLE,step000_none,solid,None,None,None,None
3,step000_01,step001_00,0,RECIRCULATING_TS,IDLE,step000_stop_recirculating_ts,solid,None,None,None,None
4,step000_01,step001_01,0,RECIRCULATING_TS,RECIRCULATING_TS,step000_none,solid,None,None,None,None


In [4]:
# Checks

# Get unique source and target node IDs from the edges DataFrame
edge_node_ids = pd.concat([edges_df['src_node_id'], edges_df['dst_node_id']]).unique()

# Get node IDs from the vertices DataFrame
vertex_node_ids = nodes_df['node_id'].unique()

# Find the difference between the two sets
missing_node_ids = set(edge_node_ids) - set(vertex_node_ids)
missing_node_ids2 = set(vertex_node_ids) - set(edge_node_ids)

print(f"Missing node IDs: {missing_node_ids}, {missing_node_ids2}")

Missing node IDs: set(), set()


## Load previously DFS generated paths
By running `recursitron.py` the paths were generated and saved in a JSON file. Now we load them and visualize them.

Done this way to avoid the performance penalty of evaluating the function inside a Jupyter notebook.

In [5]:
from solarMED_optimization import convert_to_state

# Read json file
with open('results/all_paths_SFTS.json', 'r') as f:
    all_paths = json.load(f)
    
# Convert to the correct type
all_paths = [[convert_to_state(state, state_cls=SF_TS_State) for state in path] for path in all_paths]

### Visualize the directed graph


In [7]:
from ipywidgets import interact, widgets
from solarMED_optimization.path_exploration import generate_edges_coordinates
from solarMED_optimization.visualization import plot_state_graph, get_coordinates_edge

generate_edges_coordinates(nodes_df, edges_df)

fig = plot_state_graph(nodes_df, edges_df=edges_df, system='SFTS', Np=Np)

options_avg = round(len(all_paths) / len(SF_TS_State))

@interact(initial_state=[state for state in SF_TS_State], option_idx=(0, options_avg, 1))
def add_path_highlight(initial_state=SF_TS_State.IDLE, option_idx=0):

    if initial_state is None:
        # Use random module to choose an integer from 0 to len(all_paths)
        path_idx = np.random.randint(0, len(all_paths))

    else:
        # Find the path that starts from the initial state
        path_idx = [idx for idx, path in enumerate(all_paths) if path[0] == initial_state]
        path_idx = path_idx[option_idx] if option_idx < len(path_idx) else path_idx[-1]
        print(f"Selected path {path_idx}: {[state.name for state in all_paths[path_idx]]}")

    # Somehow build the path coordinates from the list of all_paths
    path = all_paths[path_idx]

    x = []
    y = []
    for step_idx in range(0, len(path) - 1, 1):
        # If the naming scheme changes for whatever reason it will break
        src_node_id = f'step{step_idx:03d}_{path[step_idx].value}'
        dst_node_id = f'step{step_idx + 1:03d}_{path[step_idx + 1].value}'

        x_aux, y_aux = get_coordinates_edge(src_node_id, dst_node_id, nodes_df=nodes_df)

        x += x_aux
        y += y_aux

    with fig.batch_update():
        # First deactivate departing highlights
        fig.data[-2].x = None
        fig.data[-2].y = None

        # Then use arriving trace container to include the path
        fig.data[-1].x = x
        fig.data[-1].y = y

# Add a button that will highlight the paths that arrive/leave to/from a random node
# button = widgets.Button(description='Highlight random path')
# 
# button.on_click(add_path_highlight)
# 
fig
# widgets.VBox([fig, button])
# 
# add_path_highlight(None)

interactive(children=(Dropdown(description='initial_state', options=(<SF_TS_State.IDLE: '00'>, <SF_TS_State.RE…

FigureWidget({
    'data': [{'hoverinfo': 'text',
              'line': {'color': 'rgb(50,50,50)', 'width': 0.5},
              'marker': {'color': '#ff7800', 'size': 20, 'symbol': 'circle-dot'},
              'mode': 'markers',
              'name': 'states',
              'text': array(['IDLE', 'RECIRCULATING_TS', 'HEATING_UP_SF', 'SF_HEATING_TS', 'IDLE',
                             'RECIRCULATING_TS', 'HEATING_UP_SF', 'SF_HEATING_TS', 'IDLE',
                             'RECIRCULATING_TS', 'HEATING_UP_SF', 'SF_HEATING_TS', 'IDLE',
                             'RECIRCULATING_TS', 'HEATING_UP_SF', 'SF_HEATING_TS', 'IDLE',
                             'RECIRCULATING_TS', 'HEATING_UP_SF', 'SF_HEATING_TS'], dtype=object),
              'type': 'scatter',
              'uid': '824a83bb-e22a-418e-b059-ca03cf71ba6d',
              'x': array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4]),
              'y': array([0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3])

### Generate a list for every possible path
for every possible initial state

In [6]:
from recursitron import get_all_paths, dump_as
from pathlib import Path

all_paths = get_all_paths(
    system='SFTS',
    machine_init_args={
        'sample_time': 1,
    },
    max_step_idx=Np,
    initial_states=None,
    use_parallel=False
)

dump_as(all_paths, Path(f'results/all_paths_SFTS_Np_{Np}'), file_format='json')

2024-04-29 13:31:00.921 | INFO     | recursitron:generate_all_paths:124 - Hello, this is recursitron 000, current state is: SF_TS_State.IDLE, current step: 0 and we are 2 machines deep with samples [0, 0]
2024-04-29 13:31:00.921 | INFO     | recursitron:generate_all_paths:148 - Transition: start_recirculating_ts
2024-04-29 13:31:00.922 | DEBUG    | models_psa.fsms:inform_exit_state:140 - Left state IDLE
2024-04-29 13:31:00.923 | DEBUG    | models_psa.fsms:inform_enter_state:136 - Entered state RECIRCULATING_TS
2024-04-29 13:31:00.926 | INFO     | recursitron:generate_all_paths:124 - Hello, this is recursitron 001, current state is: SF_TS_State.RECIRCULATING_TS, current step: 1 and we are 3 machines deep with samples [0, 1, 1]
2024-04-29 13:31:00.930 | INFO     | recursitron:generate_all_paths:148 - Transition: stop_recirculating_ts
2024-04-29 13:31:00.930 | DEBUG    | models_psa.fsms:inform_exit_state:140 - Left state RECIRCULATING_TS
2024-04-29 13:31:00.931 | INFO     | models_psa.fsm

In [11]:
from models_psa.fsms import SolarFieldWithThermalStorage_FSM
from recursitron import get_transitions_with_inputs

model = SolarFieldWithThermalStorage_FSM(sample_time=1)
model.step(qts_src=2.0, Tsf_out=300)

model.get_inputs(format='dict')

for transition, expected_inputs in get_transitions_with_inputs(model, model.get_inputs(format='dict')):
    print(f"Transition: {transition}, Inputs: {expected_inputs}")

2024-04-25 15:10:04.968 | DEBUG    | models_psa.fsms:inform_exit_state:140 - Left state IDLE
2024-04-25 15:10:04.969 | DEBUG    | models_psa.fsms:inform_enter_state:136 - Entered state HEATING_UP_SF


Transition: stop_recirculating_sf, Inputs: {'mmed_s': None, 'mmed_f': None, 'Tmed_s_in': None, 'Tmed_c_out': None, 'med_vacuum_state': None, 'Tsf_out': 0.0}
Transition: start_sf_heating_ts, Inputs: {'mmed_s': None, 'mmed_f': None, 'Tmed_s_in': None, 'Tmed_c_out': None, 'med_vacuum_state': None, 'qts_src': 1.0, 'Tsf_out': 1.0}
Transition: none, Inputs: {'Tsf_out': 300, 'qts_src': 0}
